In [ ]:
# https://fred.stlouisfed.org/series/FEDFUNDS
# https://il.investing.com/currencies/usd-ils-historical-data
# https://www.boi.org.il/roles/markets/exchangerates/usdollar/
# https://market.tase.co.il/he/market_data/securities/data/gov_bonds
# https://market.tase.co.il/he/market_data/securities/data/bonds

# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [21]:
import numpy as np
import pandas as pd

In [22]:
finance = {
        'Finance_Id': 'Int64',
        'Year': str,
        'Month': str,
        'OBS_VALUE': str,
        'Change_From_Last_Month': str,
    }

In [49]:
# your code here
URL_ISRAEL_INTEREST_TARES = './data_Matanel/BR.csv'
URL_US_INTEREST_RATES = './data_my/FEDFUNDS.csv'
URL_USD_TO_ILS_EXCHANGE_RATES = './data_my/EXR.csv'


israel_interest_tares_csv = pd.read_csv(URL_ISRAEL_INTEREST_TARES)
israel_interest_tares = pd.DataFrame({
    'Year': [year[:4] for year in israel_interest_tares_csv['TIME_PERIOD']],
    'Month': [month[5:7] for month in israel_interest_tares_csv['TIME_PERIOD']],
    'Israel_Interest_Tares': [rate for rate in israel_interest_tares_csv['OBS_VALUE']],
})

us_interest_rates_csv = pd.read_csv(URL_US_INTEREST_RATES)
us_interest_rates = pd.DataFrame({
    'Year': [year[:4] for year in us_interest_rates_csv['DATE']],
    'Month': [month[5:7] for month in us_interest_rates_csv['DATE']],
    'Us_Interest_Rates': [rate for rate in us_interest_rates_csv['FEDFUNDS']],
})

usd_to_ils_exchange_rates_csv = pd.read_csv(URL_USD_TO_ILS_EXCHANGE_RATES)
usd_to_ils_exchange_rates = pd.DataFrame({
    'Year': [year[:4] for year in usd_to_ils_exchange_rates_csv['TIME_PERIOD']],
    'Month': [month[5:7] for month in usd_to_ils_exchange_rates_csv['TIME_PERIOD']],
    'Current_Rate': [rate for rate in usd_to_ils_exchange_rates_csv['OBS_VALUE']],
})
usd_to_ils_exchange_rates['Change_From_Last_Month'] = usd_to_ils_exchange_rates['Current_Rate'].diff()

In [24]:
# taken from: https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
data = pd.merge(israel_interest_tares, us_interest_rates, on=['Year', 'Month'], how='inner')
data = pd.merge(merged_data, usd_to_ils_exchange_rates, on=['Year', 'Month'], how='inner')

data

,Year,Month,Israel_Interest_Tares,Us_Interest_Rates,Current_Rate,Change_From_Last_Month
0,1994,01,10.50,3.05,2.986,0.000
1,1994,01,10.50,3.05,2.986,0.000
2,1994,01,10.50,3.05,2.995,0.009
3,1994,01,10.50,3.05,2.987,-0.008
4,1994,01,10.50,3.05,2.987,0.000
...,...,...,...,...,...,...
227434,2023,12,4.75,5.33,3.599,-0.017
227435,2023,12,4.75,5.33,3.628,0.029
227436,2023,12,4.75,5.33,3.624,-0.004
227437,2023,12,4.75,5.33,3.619,-0.005


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [25]:
# A - 1
# your code here
us_interest_rates.head(5)

,Year,Month,Us_Interest_Rates
0,1954,07,0.80
1,1954,08,1.22
2,1954,09,1.07
3,1954,10,0.85
4,1954,11,0.83


In [26]:
# A - 2
# your code here
israel_interest_tares['Israel_Interest_Tares'].mean()

4.880738267313146

In [27]:
# A - 3
# your code here
usd_to_ils_exchange_rates['Change_From_Last_Month'].max()

0.21499999999999986

In [35]:
# A - 4
# your code here
us_interest_rates['Change_From_Last_Month'] = us_interest_rates['Us_Interest_Rates'].diff()

(us_interest_rates['Change_From_Last_Month'] < 0).sum()

328

In [39]:
# A - 5
# your code here
israel_interest_tares['Change_From_Last_Month'] = israel_interest_tares['Israel_Interest_Tares'].diff()

israel_interest_tares['Change_Percentage'] = (israel_interest_tares['Change_From_Last_Month'] / israel_interest_tares['Israel_Interest_Tares'].shift(1)) * 100

israel_interest_tares['Change_Percentage'].max()

249.99999999999994

## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [50]:
# B - 1
# your code here
merged_data = pd.merge(israel_interest_tares, us_interest_rates, on=['Year', 'Month'], how='inner')[['Year', 'Month', 'Israel_Interest_Tares', 'Us_Interest_Rates']]

merged_data

,Year,Month,Israel_Interest_Tares,Us_Interest_Rates
0,1994,01,10.50,3.05
1,1994,01,10.50,3.05
2,1994,01,10.50,3.05
3,1994,01,10.50,3.05
4,1994,01,10.50,3.05
...,...,...,...,...
10926,2023,12,4.75,5.33
10927,2023,12,4.75,5.33
10928,2023,12,4.75,5.33
10929,2023,12,4.75,5.33


In [53]:
# B - 2
# your code here
# אחרי שראיתי את שאלה B5
# הבנתי מה צריך לעשות
merged_data['Rate_Difference'] = ((1 + all_data['Israel_Interest_Tares']) / (1 + all_data['Us_Interest_Rates']) - 1)
merged_data['Rate_Difference']

0        1.839506
1        1.839506
2        1.839506
3        1.839506
4        1.839506
           ...   
10926    1.241135
10927    1.241135
10928    1.241135
10929    1.241135
10930    1.241135
Name: Rate_Difference, Length: 10931, dtype: float64

In [46]:
# B - 3
# your code here

# https://www.calculatorsoup.com/calculators/algebra/percentage-increase-calculator.php
conditions = [
    (merged_data['Israel_Interest_Tares'] >= merged_data['Us_Interest_Rates'] + 1),
    (merged_data['Us_Interest_Rates'] >= merged_data['Israel_Interest_Tares'] + 1),
    (abs(merged_data['Israel_Interest_Tares'] - merged_data['Us_Interest_Rates']) < 1)
]

choices = ['IH', 'UH', 'N']

merged_data['Rate_Comparison'] = np.select(conditions, choices, default='Unknown')
merged_data['Rate_Comparison']

0        IH
1        IH
2        IH
3        IH
4        IH
         ..
10926     N
10927     N
10928     N
10929     N
10930     N
Name: Rate_Comparison, Length: 10931, dtype: object

In [47]:
# B - 4
# your code here
all_data = pd.merge(usd_to_ils_exchange_rates, merged_data, on=['Year', 'Month'], how='inner')
all_data

,Year,Month,Current_Rate,Change_From_Last_Month,Israel_Interest_Tares,Us_Interest_Rates,Rate_Difference,Rate_Comparison
0,1994,01,2.986,0.000,10.50,3.05,7.45,IH
1,1994,01,2.986,0.000,10.50,3.05,7.45,IH
2,1994,01,2.986,0.000,10.50,3.05,7.45,IH
3,1994,01,2.986,0.000,10.50,3.05,7.45,IH
4,1994,01,2.986,0.000,10.50,3.05,7.45,IH
...,...,...,...,...,...,...,...,...
227434,2023,12,3.627,0.008,4.75,5.33,-0.58,N
227435,2023,12,3.627,0.008,4.75,5.33,-0.58,N
227436,2023,12,3.627,0.008,4.75,5.33,-0.58,N
227437,2023,12,3.627,0.008,4.75,5.33,-0.58,N


In [55]:
# B - 5
# your code here

# https://www.globes.co.il/news/article.aspx?did=1001489717
# https://tradingpedia.co.il/%D7%94%D7%92%D7%93%D7%A8%D7%94-%D7%A0%D7%95%D7%A1%D7%97%D7%94-%D7%95%D7%93%D7%95%D7%92%D7%9E%D7%94-%D7%A9%D7%9C-%D7%A9%D7%95%D7%95%D7%99-%D7%A8%D7%99%D7%91%D7%99%D7%AA-%28irp%29/
# https://www.investopedia.com/terms/i/interestrateparity.asp

# F0=S0×(1+ic1+ib) זה נוסכה שמצאתי באיטרנט על חישוב שוויון ריבית הסבר על מה זה שוויון ריבית מוצג בקישור
# כמו שהבנתי צריך לחשב את זה ככה (שיניתי את שאלה 2 ע"פ הפונקציה)
IRP = all_data['Rate_Difference'] * all_data['Current_Rate']
IRP

0         22.24570
1         22.24570
2         22.24570
3         22.24570
4         22.24570
            ...   
227434    -2.10366
227435    -2.10366
227436    -2.10366
227437    -2.10366
227438    -2.10366
Length: 227439, dtype: float64

## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [76]:
# C - 1
# your code here

all_data[(all_data['Us_Interest_Rates'].diff() > 0) & (all_data['Change_From_Last_Month'] > 0)].count() # 68
all_data[(all_data['Us_Interest_Rates'].diff() > 0)].count() # 173

# פחות מחמישים אחוז אז נראה שזה לא קשור

Year                      173
Month                     173
Current_Rate              173
Change_From_Last_Month    173
Israel_Interest_Tares     173
Us_Interest_Rates         173
Rate_Difference           173
Rate_Comparison           173
dtype: int64

In [77]:
# C - 2
# your code here
all_data[(all_data['Israel_Interest_Tares'].diff() > 0) & (all_data['Change_From_Last_Month'] > 0)].count() # 1213

# נראה שיש

Year                      1213
Month                     1213
Current_Rate              1213
Change_From_Last_Month    1213
Israel_Interest_Tares     1213
Us_Interest_Rates         1213
Rate_Difference           1213
Rate_Comparison           1213
dtype: int64

In [ ]:
# C - 3
# your code here


In [ ]:
# C - 4
# your code here


In [78]:
# C - 5
# your code here
all_data[(all_data['Israel_Interest_Tares'].diff() > 0) & (all_data['Us_Interest_Rates'] > 0)]['Current_Rate'].mean()

3.8242272727272724

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here